<a href="https://colab.research.google.com/github/juan-orea/03MAIR-Algoritmos-de-optimizacion/blob/master/AG2/AG2_Juan_Orea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Juan Manuel Orea - AG2<br>

Actividad guiada 2<br>
url: https://github.com/juan-orea/03MAIR-Algoritmos-de-optimizacion/blob/master/AG2/Juan_Orea_AG2.ipynb

In [0]:
# Función auxiliar (decorator) para medir tiempos de ejecución de una función (propuesto por Isabel)
from time import time
#Función para calcular el tiempo de ejecución
def calcular_tiempo(f):
    def wrapper(*args, **kwargs):      
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper
from time import time

In [0]:
import random
import math

In [0]:
def distancia(A, B):
  if type(A) is int or type(A) is float:
    return abs(B - A)
  if A == None or B == None:
    return float("infinity")
  else:
    return math.sqrt(sum([ (A[i] - B[i])**2 for i in range(len(A))  ]))

In [0]:
# Aproximacion por fuerza bruta
# @calcular_tiempo
def distancia_FuerzaBruta(L):
  # resultado parcial (acumulado según el progreso): distancia y puntos para los que se dan
  distancia_minima = float("infinity")  #  inicializamos a inf para asegurar que vamos actualizando con nuestros valores
  A, B = None, None
  
  for i in range(len(L)):
    for j in range(i+1, len(L)):
      dist = distancia(L[i], L[j])  # distancia 'local': la de los dos puntos en observacion
      if dist < distancia_minima:
        A, B = L[i], L[j]
        distancia_minima = dist

  if distancia_minima == float("infinity"):  # significa que no se ha podido calcular ninguna distancia (la lista está vacía o solo hay un elemento) 
    return []
  else:
    return [A, B]


In [0]:
# Algunas pruebas de ejecucion: medimos tiempos para distintos tamaños de lista y dimensiones
@calcular_tiempo
def midiendo_distancia_FuerzaBruta(L):
  return distancia_FuerzaBruta(L)

for i in  [1, 2, 3]:           # dimensiones
  print('Dimensiones: {0}'.format(i))
  for N in [100, 1000, 3000]:  # tamaño del conjunto de prueba
    if i == 1:
      LISTA = [ random.randrange(1, N * 10) for _ in range(N) ]
    elif i == 2:
      LISTA = [ (random.randrange(1, N * 10), random.randrange(1, N * 10)) for _ in range(N) ]
    else:
      LISTA = [ (random.randrange(1, N * 10), random.randrange(1, N * 10), random.randrange(1, N * 10)) for _ in range(N) ]
    print('Prueba para {0} elementos, resultado: {1}'.format(N, midiendo_distancia_FuerzaBruta(LISTA)))

Comprobamos empíricamente que el orden es cuadrático.<br>
Comprobamos que funciona correctamente para distinto número de dimensiones (más alla de 2 no hay diferencia realmente, así que llegamos solo hasta 3D).

In [0]:
# Aproximacion por estrategia de divide y venceras
# para calcular una métrica escalar de forma homogénea tanto en tuplas como en valores de naturaleza escalar
# para tuplas simplemente se devuelve el primer elemento (primera dimension)
def valor(v):
  if type(v) is int or type(v) is float:
    return v
  else:
    return v[0]

# para calcular un punto intermedio de una lista dada: se utiliza promedia sobre un número dado de elementos
# los valores se cogen de forma aleatoria para mitigar problemas en listas ordenadas (por ejemplo)
def calcularPivote(L, N):
  largoLista = len(L)
  sumaParcial = 0
  if len(L) == 0: return 0

  for i in range(N):
      sumaParcial += valor(L[random.randrange(0, largoLista)])

  return sumaParcial / N


def distancia_divide_y_venceras(L):
  #Si hay pocos por Fuerza Bruta (se podría probar con distintos valores para decidir qué es "poco")
  if len(L) < 10: 
    return  distancia_FuerzaBruta(L)
  
  # Dividir para listas grandes
  pivote = calcularPivote(L, 10)
  LISTA_IZQ, LISTA_DER, LISTA_CENTRO = [], [], []
  for punto in L:
    if valor(punto) < pivote:
      LISTA_IZQ.append(punto)
    else:
      LISTA_DER.append(punto)
  
  # Lista_ordenada = sorted(L, key=lambda x: x[0])
  # LISTA_IZQ = Lista_ordenada[:len(L)//2]
  # LISTA_DER = Lista_ordenada[len(L)//2:]
  
  PUNTOS_LISTA_IZQ = distancia_divide_y_venceras(LISTA_IZQ)
  PUNTOS_LISTA_DER = distancia_divide_y_venceras(LISTA_DER)
  
  # Para cubrir todas las casuísticas hay que comprobar los puntos "centrales" (alrededor del punto / línea de corte: C)
  # tenemos en cuenta que el caso más desfavorable sería tener un punto A justo en el límite cerca de otro B que está del otro lado del corte
  # este punto B debería estar respecto del punto de corte a una distancia menor que la menor distancia que hemos encontrado hasta ahora: d
  # por tanto buscamos puntos que estén entre C - d y C + d: cogemos los puntos que queden en ese intervalo y les aplicamos también la búsqueda
  d1 = distancia(PUNTOS_LISTA_IZQ[0], PUNTOS_LISTA_IZQ[1]) if len(PUNTOS_LISTA_IZQ) > 1 else float("infinity")
  d2 = distancia(PUNTOS_LISTA_DER[0], PUNTOS_LISTA_DER[1]) if len(PUNTOS_LISTA_DER) > 1 else float("infinity")
  d = d1 if d1 < d2 else d2
  for punto in L:
    if valor(punto) > pivote - d and valor(punto) < pivote + d:  # consideramos solo menor (y mayor) estricto, ya que la igualdad no mejoraría la menor distancia
      LISTA_CENTRO.append(punto)
  PUNTOS_LISTA_CENTRO = distancia_FuerzaBruta(LISTA_CENTRO)
 
  # evitamos duplicar algún punto (haría que saliese esto como la menor distancia)
  if len(PUNTOS_LISTA_CENTRO) > 1 and (PUNTOS_LISTA_CENTRO[1] in PUNTOS_LISTA_IZQ or PUNTOS_LISTA_CENTRO[1] in PUNTOS_LISTA_DER):
    del PUNTOS_LISTA_CENTRO[1]
  if len(PUNTOS_LISTA_CENTRO) > 0 and (PUNTOS_LISTA_CENTRO[0] in PUNTOS_LISTA_IZQ or PUNTOS_LISTA_CENTRO[0] in PUNTOS_LISTA_DER):
    del PUNTOS_LISTA_CENTRO[0]

  
  return distancia_FuerzaBruta(PUNTOS_LISTA_IZQ + PUNTOS_LISTA_DER + PUNTOS_LISTA_CENTRO)

In [0]:
# Algunas pruebas de ejecucion: medimos tiempos para distintos tamaños de lista y dimensiones
@calcular_tiempo
def midiendo_distancia_divide_y_venceras(L):
  return distancia_divide_y_venceras(L)

for i in  [1, 2, 3]:           # dimensiones
  print('Dimensiones: {0}'.format(i))
  for N in [1000, 10000, 30000]:  # tamaño del conjunto de prueba
    if i == 1:
      LISTA = [ random.randrange(1, N * 10) for _ in range(N) ]
    elif i == 2:
      LISTA = [ (random.randrange(1, N * 10), random.randrange(1, N * 10)) for _ in range(N) ]
    else:
      LISTA = [ (random.randrange(1, N * 10), random.randrange(1, N * 10), random.randrange(1, N * 10)) for _ in range(N) ]
    print('Prueba para {0} elementos, resultado: {1}'.format(N, midiendo_distancia_divide_y_venceras(LISTA)))

He incluído una variación sobre lo visto en clase, evitando la ordenación de la lista para hacer las mitades. La ordenación que hace Python es muy eficiente, prácticamente O(N) en algunas pruebas rápidas que he hecho (supongo que usa estructuras tipo hash o similar), de forma que con toda seguridad será una ventaja hacer la ordenación para asegurar que se van cogiendo mitades iguales.<br>
Sin embargo me ha parecido que en este caso era "mezclar", que en todo caso se podría haber hecho una ordenación inicial y trabajar con la lista ya ordenada.<br>
Comprobamos que funciona correctamente para distinto número de dimensiones, sin embargo vemos que ya para 3 dimensiones se comporta bastante mal en cuanto a rendimiento. Dado que no supone gran diferencia computacional trabajar en N dimensiones (a partir de N > 1, lo podemos ver para las pruebas de fuerza bruta), creo que el problema viene por el tratamiento de la "*zona de contacto*", ya que va empeorando la relación entre la dimensión de particionado y el resto de dimensiones.<br>
Seguro que en este punto hay bastante margen para mejorar.

In [0]:
# Paseo por el rio: calculo de la forma más eficiente para ir de un punto del río a otro, en base a la tabla de tarifas
def mejoresPrecios(T):
  n_nodos = len(T[0])
  
  PRECIOS = [  [9999] * n_nodos for i in range(n_nodos) ]
  RUTAS   = [  ['']   * n_nodos for i in range(n_nodos) ]

  for i in range(n_nodos - 1):
    for j in range(i + 1, n_nodos):
      # inicializacion: el viaje directo
      precioViaje = T[i][j]   # el precio de ir de i a j es la tarifa directa de i a j
      RUTAS[i][j] = i         # llegamos a j desde i
      
      # ahora buscamos combinaciones con trasbordo intermedio
      for k in range(i, j):
        PrecioConTrasbordo = PRECIOS[i][k] + T[k][j]
        if PrecioConTrasbordo < precioViaje:
          precioViaje = PrecioConTrasbordo
          RUTAS[i][j] = k         # llegamos a j desde k (saliendo de i)

      # Finalmente se fija el precio mejor en la tabla de resultados (se podría haber trabajado directamente sobre este elemento)
      PRECIOS[i][j] = precioViaje
          
  return PRECIOS, RUTAS

# Función auxiliar para "aplanar" las posibles iteraciones (trasbordos)
def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    return desde 

  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ' --> ' + str(hasta)


TARIFAS = [
           [0,   5,   4,   3,   999, 999, 999],
           [999, 0,   999, 2,   3,   999, 11],
           [999, 999, 0,   1,   999, 4,   10],
           [999, 999, 999, 0,   5,   6,   9],
           [999, 999, 999, 999, 0,   999, 4],
           [999, 999, 999, 999, 999, 0,   3],
           [999, 999, 999, 999, 999, 999, 0]
          ]

p, r = precio(TARIFAS)

print(r)
print()
print("La ruta es:")  
calcular_ruta(r, 1, 6)
